# MODEL TRAINING

### 1.1) Import Data and Required Packages

In [3]:
!pip install scikit-learn

  Using cached scikit_learn-1.3.2-cp38-cp38-macosx_10_9_x86_64.whl.metadata (11 kB)
  Using cached scipy-1.10.1-cp38-cp38-macosx_10_9_x86_64.whl.metadata (53 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.3.2-cp38-cp38-macosx_10_9_x86_64.whl (10.1 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached scipy-1.10.1-cp38-cp38-macosx_10_9_x86_64.whl (35.0 MB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)


In [4]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Modelling
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.model_selection import RandomizedSearchCV
import warnings

#### Import the CSV Data as Pandas DataFrame

In [5]:
df = pd.read_csv('data/stud.csv')

In [6]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [7]:
X = df.drop(columns=['math_score'],axis=1)

In [8]:
y = df['math_score']

In [10]:
# Create Column Transformer with 3 types of transformers
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(exclude="object").columns

from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer,cat_features),
        ("StandardScaler", numeric_transformer,num_features)
    ]
)

In [11]:
X = preprocessor.fit_transform(X)
X.shape

(1000, 151)

In [14]:
# separate dataset into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape

((800, 151), (200, 151))

### Create Evaluation Function to give all metrics after model Training

In [17]:
def evaluate_model(true,predicted):
    mae = mean_absolute_error(true,predicted)
    mse = mean_squared_error(true,predicted)
    rmse = np.sqrt(mean_squared_error(true,predicted))
    r2 = r2_score(true,predicted)
    return mae, rmse, r2

In [18]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),

}
model_list = []
r2_list = []

for i in range(len(list(models))):
    # Train Model
    model = list(models.values())[i]
    model.fit(X_train,y_train)

    # Make Predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Evaluate Model
    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(y_train,y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test,y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model performance for Training set')
    print('- Root Mean Square Error: {:.4f}'.format(model_train_rmse))
    print('- Mean Absolute Error: {:.4f}'.format(model_train_mae))
    print('- R2 score: {:.4f}'.format(model_train_r2))

    print('----------------------------------------')
    
    print('Model performance for Testing set')
    print('- Root Mean Square Error: {:.4f}'.format(model_test_rmse))
    print('- Mean Absolute Error: {:.4f}'.format(model_test_mae))
    print('- R2 score: {:.4f}'.format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')
    



Linear Regression
Model performance for Training set
- Root Mean Square Error: 7.8301
- Mean Absolute Error: 6.2872
- R2 score: 0.7280
----------------------------------------
Model performance for Testing set
- Root Mean Square Error: 9.7757
- Mean Absolute Error: 7.9516
- R2 score: 0.6073


Lasso
Model performance for Training set
- Root Mean Square Error: 8.6808
- Mean Absolute Error: 6.9821
- R2 score: 0.6658
----------------------------------------
Model performance for Testing set
- Root Mean Square Error: 8.8751
- Mean Absolute Error: 7.3635
- R2 score: 0.6763


Ridge
Model performance for Training set
- Root Mean Square Error: 7.8737
- Mean Absolute Error: 6.3800
- R2 score: 0.7250
----------------------------------------
Model performance for Testing set
- Root Mean Square Error: 9.3038
- Mean Absolute Error: 7.6834
- R2 score: 0.6443


K-Neighbors Regressor
Model performance for Training set
- Root Mean Square Error: 7.4692
- Mean Absolute Error: 6.0132
- R2 score: 0.7525
---

## RESULTS

In [23]:
pd.DataFrame(list(zip(model_list,r2_list)), columns=['Model Name', 'r2_score']).sort_values(by=['r2_score'],ascending=False)

,Model Name,r2_score
1,Lasso,0.676304
2,Ridge,0.644280
0,Linear Regression,0.607276
3,K-Neighbors Regressor,0.594924
5,Random Forest Regressor,0.562452
4,Decision Tree,0.436363
